## RQ1 Creating FAO forest map

This script is focused on creating one of the forest products (the FAO-aligned definition) for comparison with the other forest products as part of **RQ1: What is considered to be forest?** The steps generally follow the approach taken in Johnson et al (2023).

As a bit of background for the GER LULC class numbers:
- Class 3 corresponds to mainly in-land natural land covers: forest, grasslands, moors/heathland, transitional woodland/shrub, beaches/dunes, bare rock, sparsely vegetated areas, burnt areas and glaciers or perpetual snow
- Class 4 corresponds to coastal natural land covers: inland marshes, peatbogs, coastal salt marshes and intertidal flats
- the remaining classes correspond to urban areas (Class 1), agricultural areas (Class 2) and water bodies (Class 5)

Steps:
1. Rasterise GER LULC Class 3 & 4
2. Mask JAXA FNF with GER LULC Class 3 & 4
3. Combine adjusted JAXA FNF with GER LULC FNF
4. Copy & rename

In [ ]:
# SETUP

# Note: this .ipynb file depends on files & folder structures created in rq1_step1_data_prep.ipynb

# Import packages
import os
import subprocess
import shutil

import pandas as pd
import geopandas as gpd
import rasterio

# Store gdal.exe paths
gdal_rasterize = "./thesis_env_conda/Library/bin/gdal_rasterize.exe"
gdalwarp = "./thesis_env_conda/Library/bin/gdalwarp.exe"

# Store gdal.py paths
gdal_calc = "./thesis_env_conda/Lib/site-packages/GDAL-3.10.1-py3.12-win-amd64.egg-info/scripts/gdal_calc.py"


### Step 1: Rasterise GER LULC Class 3 & 4

Originally I planned to use a vector of the GER LULC Class 3 and 4 as a clipper to extract only those areas from the JAXA FNF raster. However, this resulted in a very complex vector and the clipping process was prohibitively slow. 

Instead I decided to rasterise the GER LULC 3 and 4 Classes which correspond to natural areas (value of 1 for these classes and 0 everywhere else). I can then use this raster to remove the non-class-3 and non-class-4 (urban and agricultural areas plus water bodies) areas from the JAXA FNF map by multiplying the two together. 

Note the dissolve and explode steps are not totally needed (previously they were required for creating a clipper shp).

In [ ]:
# 1: COMBINE GER LULC SHPS
# TAKES ABOUT 15 MIN

# Load all the GER LULC SHPs (no data from the attribute table is needed)
ger_lulc_class3_shp = gpd.read_file("./processing/clc5_class3xx_3035_DE.shp", columns = [""])
ger_lulc_class4_shp = gpd.read_file("./processing/clc5_class4xx_3035_DE.shp", columns = [""])

# Append the shapefiles together
merged_ger_lulc_shp = pd.concat([ger_lulc_class3_shp,
                                 ger_lulc_class4_shp,
                                 ])

# Dissolve the geometries (this creates one multi-part geometry)
dissolved_ger_lulc_shp = merged_ger_lulc_shp.dissolve()

# Explode the multi-part geometry into multiple single geometries
exploded_ger_lulc_shp = dissolved_ger_lulc_shp.explode()

# Write the merged, dissolved & exploded output to file
exploded_ger_lulc_shp.to_file('./processing/clc5_class3_class4_3035_DE.shp')


In [2]:
# 1: RASTERISATION
# ABOUT 25 MIN

# Store path to shp for rasterising
gerlulc_3_4_shp = "./processing/clc5_class3_class4_3035_DE.shp"

# Store path/filename for rasterised output
gerlulc_3_4_tif = "./processing/clc5_class3_class4_3035_DE_5m.tif"

# Run gdal_rasterize to create GER LULC Class 3 & 4 raster
# I removed the inversion as this caused it to run for hours without producing data (1 KB file only)
# '-i' flag: invert rasterisation (burn value is burned into all parts NOT inside the polygons)
gerlulc_rasterise = subprocess.run([gdal_rasterize, 
                                    '-l', 'clc5_class3_class4_3035_DE',
                                    '-burn', '1',
                                    #'-i',    
                                    '-tr', '5', '5',
                                    '-a_nodata', '-9999', 
                                    '-ot', 'Int16', 
                                    '-of', 'GTiff',
                                    '-co', 'COMPRESS=LZW', 
                                    '-co', 'BIGTIFF=YES', 
                                    gerlulc_3_4_shp,
                                    gerlulc_3_4_tif
                                    ],
                                    capture_output=True, 
                                    text=True)

print(gerlulc_rasterise.stdout)
print(gerlulc_rasterise.stderr)


0...10...20...30...40...50...60...70...80...90...100 - done.




In [ ]:
# 1: CONVERT NODATA TO VALID VALUE 
# Block/window processing is needed to work with the data in chunks, otherwise I hit memory problems!

# Input file
gerlulc_3_4_tif = "./processing/clc5_class3_class4_3035_DE_5m.tif"

# Output file
gerlulc_3_4_fix = "./processing/clc5_class3_class4_3035_DE_5m_reclass.tif"

# Open input raster
with rasterio.open(gerlulc_3_4_tif) as input:
    # Copy profile (metadata) from input
    profile = input.profile.copy()  
    # Update profile (metdata) for output
    profile.update(dtype=rasterio.int16, nodata=-9999, compress="LZW")  

    # Create the output using the updated profile (metadata)
    with rasterio.open(gerlulc_3_4_fix, 'w', **profile) as output:
        # Process each block one at a time 
        for ji, window in input.block_windows(1):     # the 1 here indicates band 1
            # Store the data for a block
            data = input.read(1, window=window)       # the 1 here indicates band 1 
            # Replace NoData values (-9999) with 0 for a block
            data[data == -9999] = 0
            # Write the adjusted data for a block to the output raster
            output.write(data, 1, window=window) 


### Step 2: Mask JAXA FNF with GER LULC Class 3 & 4

Using the raster from step 1, the next step is to adjust the JAXA FNF map so that all non-class-3 and non-class-4 forests are removed. This can be achieved by multiplying the rasters, so that non-class-3 and non-class-4 areas are converted to 0. This removes any areas of forest in the JAXA map which are in predominantly agricultural or urban areas.

Before I multiply the two layers, the extents will need to match - so here I follow a similar process as in "rq1_step1_data_prep.ipynb" to clip and adjust the extents of the GER LULC Class 3 & 4 raster.

In [6]:
# 2: CLIP TO CORINE BBOX
# TAKES ABOUT 33 MIN

# Store path to CORINE bbox shp (created in "rq1_step1_data_prep.ipynb")
corine_bbox_shp = "./processing/corine_reclass_bbox.shp"

# Define function that clips tifs to the CORINE bbox (copy from "rq1_step1_data_prep.ipynb")
def bbox_clip(input_paths):
    # Iterate through the paths 
    for path in input_paths:
        # For output file naming: extract the input file name (with extension)
        name_w_ext = os.path.split(path)[1] 
        # For output file naming: remove extension
        root_name = name_w_ext[:-4]
        # For output file naming: assemble the new file path for the output
        output_path = "./processing/" + root_name + "_bboxclip.tif"

        # Run warp to crop to the CORINE bbox
        clip_to_bbox = subprocess.run([gdalwarp, 
                                       '-crop_to_cutline', 
                                       '-cutline', corine_bbox_shp, 
                                       '-tr', '5', '5',
                                       '-dstnodata', '-9999', 
                                       '-ot', 'Int16', 
                                       '-co', 'COMPRESS=LZW', 
                                       '-co', 'BIGTIFF=YES', 
                                       path, 
                                       output_path
                                       ],
                                       capture_output=True, 
                                       text=True)
        print(clip_to_bbox.stdout)
        print(clip_to_bbox.stderr)

# Run just for the single raster
bbox_clip(["./processing/clc5_class3_class4_3035_DE_5m_reclass.tif"])

Creating output file that is 128214P x 173470L.
Using internal nodata values (e.g. -9999) for image ./processing/clc5_class3_class4_3035_DE_5m_reclass.tif.
Processing ./processing/clc5_class3_class4_3035_DE_5m_reclass.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.




In [7]:
# 2: WARP EXTENTS
# TAKE ABOUT 17 MIN

# First, extract the extents from the CORINE data (created in "rq1_step1_data_prep.ipynb")
corine_ref = rasterio.open("./processing/U2018_CLC2018_V2020_3035_DE_5m_bboxclip.tif")
corine_bounds  = corine_ref.bounds

# Store the bounds in the format required for gdalwarp
corine_xmin = str(corine_bounds[0])       # xmin = left
corine_ymin = str(corine_bounds[1])       # ymin = bottom
corine_xmax = str(corine_bounds[2])       # xmax = right
corine_ymax = str(corine_bounds[3])       # ymax = top

# Define function that warps rasters to the CORINE extents (copy from "rq1_step1_data_prep.ipynb")
def corine_warp(input_paths):
    # Iterate through the paths 
    for path in input_paths:
        # For output file naming: extract the input file name (with extension)
        name_w_ext = os.path.split(path)[1] 
        # For output file naming: remove extension from input file name
        name_wo_ext = os.path.splitext(name_w_ext)[0]
        # For output file naming: assemble the new file path for the output
        output_path = "./processing/" + name_wo_ext + "_warp_exts.tif"
        
        # Run warp to match all rasters to CORINE extents
        warp_extents = subprocess.run([gdalwarp, 
                                      '-t_srs', 'EPSG:3035', 
                                      #'-tr', '5', '5',
                                      '-te', corine_xmin, corine_ymin, corine_xmax, corine_ymax,
                                      #'-tap',
                                      '-ot', 'Int16', 
                                      '-co', 'COMPRESS=LZW', 
                                      '-co', 'BIGTIFF=YES', 
                                      path, 
                                      output_path
                                      ],
                                      capture_output=True, 
                                      text=True)
        print(warp_extents.stdout)
        print(warp_extents.stderr)


# Separate Hansen processing
corine_warp(["./processing/clc5_class3_class4_3035_DE_5m_reclass_bboxclip.tif"])

Creating output file that is 128214P x 173470L.
Using internal nodata values (e.g. -9999) for image ./processing/clc5_class3_class4_3035_DE_5m_reclass_bboxclip.tif.
Copying nodata values from source ./processing/clc5_class3_class4_3035_DE_5m_reclass_bboxclip.tif to destination ./processing/clc5_class3_class4_3035_DE_5m_reclass_bboxclip_warp_exts.tif.
Processing ./processing/clc5_class3_class4_3035_DE_5m_reclass_bboxclip.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.




In [4]:
# 2: MULTIPLY RASTERS
# TAKES ABOUT 8 MIN

# Store paths to the two input maps
jaxa_FNF = "./processing/jaxa_FNF_3035_DE_5m_bboxclip_warp_exts.tif" # matches extents of adjuster!
ger_lulc_adjuster = "./processing/clc5_class3_class4_3035_DE_5m_reclass_bboxclip_warp_exts.tif"

# Runs gdal_calc.py to subtract the input rasters  
adjusted_jaxa = subprocess.run(['python', 
                                gdal_calc, 
                                '-A', jaxa_FNF, 
                                '-B', ger_lulc_adjuster, 
                                '--outfile=./processing/jaxa_3035_DE_5m_adjusted.tif', 
                                '--calc=A*B', 
                                '--co=COMPRESS=LZW', 
                                '--co=BIGTIFF=YES', 
                                '--NoDataValue=-9999'
                                ],
                                capture_output=True, 
                                text=True)

print(adjusted_jaxa.stdout)
print(adjusted_jaxa.stderr)

0...10...20...30...40...50...60...70...80...90...100 - done.




### Step 3: Combine adjusted JAXA FNF with GER LULC FNF

The JAXA map now meets the FAO requirements so it can be added together with the GER LULC FNF map (which also is within the FAO definition thresholds).

The output from adding the two maps together will include values 0 to 2, so the final step is to convert the map back into a FNF output (with only values of 0 and 1).

In [ ]:
# 3: COMBINE ADJUSTED JAXA & GER LULC
# TAKES ABOUT 10 MIN

# Store paths to the two input maps
adjusted_jaxa_FNF = "./processing/jaxa_3035_DE_5m_adjusted.tif"
ger_lulc_FNF = "./processing/clc5_class3xx_3035_DE_5m_bboxclip_warp_exts.tif" # matches extents!

# Runs gdal_calc.py to add the input rasters together 
initial_fao = subprocess.run(['python', 
                              gdal_calc, 
                              '-A', adjusted_jaxa_FNF, 
                              '-B', ger_lulc_FNF, 
                              '--outfile=./processing/fao_approx_3035_DE_5m_calc.tif', 
                              '--calc=A+B', 
                              '--co=COMPRESS=LZW', 
                              '--co=BIGTIFF=YES', 
                              '--NoDataValue=-9999'
                              ],
                              capture_output=True, 
                              text=True)

print(initial_fao.stdout)
print(initial_fao.stderr)

0...10...20...30...40...50...60...70...80...90...100 - done.




In [7]:
# 3: RECLASSIFY TO FNF
# TAKES ABOUT 10 MIN

# Store path to initial FAO map
initial_fao = "./processing/fao_approx_3035_DE_5m_calc.tif"

# Runs gdal_calc.py in order to reclassify to FNF
reclass_fao = subprocess.run(['python', 
                              gdal_calc, 
                              '-A', initial_fao, 
                              '--outfile=./processing/fao_approx_3035_DE_5m_calc_reclass.tif', 
                              '--calc=-9999*(A==-9999)+0*(A==0)+1*(A==1)+1*(A==2)', 
                              '--co=COMPRESS=LZW', 
                              '--co=BIGTIFF=YES', 
                              '--NoDataValue=-9999'
                              ],
                              capture_output=True, 
                              text=True)

print(reclass_fao.stdout)
print(reclass_fao.stderr)

0...10...20...30...40...50...60...70...80...90...100 - done.




In [8]:
# 3: CLIP TO CORINE FOOTPRINT
# TAKES ABOUT 170 MIN

# Store path to the input map 
reclass_fao = "./processing/fao_approx_3035_DE_5m_calc_reclass.tif"

# Store path to CORINE footprint (created in "rq1_step1_data_prep.ipynb")
corine_clipper = "./processing/clipper.shp"

# Store path to output
clipped_fao = "./processing/fao_approx_3035_DE_5m_calc_reclass_clipped.tif"

# Use gdalwarp to clip tif to the CORINE footprint
clip_to_DE = subprocess.run([gdalwarp, 
                             '-crop_to_cutline', 
                             '-cutline', corine_clipper, 
                             '-tr', '5', '5',
                             '-dstnodata', '-9999', 
                             '-ot', 'Int16', 
                             '-co', 'COMPRESS=LZW', 
                             '-co', 'BIGTIFF=YES', 
                             reclass_fao, 
                             clipped_fao                    
                             ],
                             capture_output=True,           
                             text=True)
         
print(clip_to_DE.stdout)
print(clip_to_DE.stderr)

Creating output file that is 128212P x 173469L.
Using internal nodata values (e.g. -9999) for image ./processing/fao_approx_3035_DE_5m_calc_reclass.tif.
Processing ./processing/fao_approx_3035_DE_5m_calc_reclass.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.




### Step 4: Copy & Rename

After visually checking the raster in QGIS, the output from the last step seems to meet all the requirments! I now copy over the raster to the "outputs" folder and rename it to indicate it is the final FNF output. 

In [9]:
# 4: COPY & RENAME

# Store the path to the old version (to be copied & renamed)
fao_old = "./processing/fao_approx_3035_DE_5m_calc_reclass_clipped.tif"

# Copy & rename the raster
shutil.copy(fao_old, "./outputs/" + os.path.split(fao_old)[1][:-24] + "2018_FNF.tif")

'./outputs/fao_approx_3035_DE_5m_2018_FNF.tif'

**Citations:**

Johnson, B. A., Umemiya, C., Magcale-Macandog, D. B., Estoque, R. C., Hayashi, M., & Tadono, T. (2023). Better monitoring of forests according to FAO’s definitions through map integration: Significance and limitations in the context of global environmental goals. *International Journal of Applied Earth Observation and Geoinformation, 122,* 103452. https://doi.org/10.1016/j.jag.2023.103452